In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110520  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,9.341205e+06
2018-02-28,8.564947e+06
2018-03-31,8.714933e+06
2018-04-30,8.264466e+06
2018-05-31,8.660008e+06
2018-06-30,8.276295e+06
2018-07-31,9.233724e+06
2018-08-31,8.706657e+06
2018-09-30,8.551479e+06


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    9.341205e+06
 2018-02-28    8.564947e+06
 2018-03-31    8.714933e+06
 2018-04-30    8.264466e+06
 2018-05-31    8.660008e+06
 2018-06-30    8.276295e+06
 2018-07-31    9.233724e+06
 2018-08-31    8.706657e+06
 2018-09-30    8.551479e+06
 2018-10-31    8.330309e+06
 2018-11-30    8.079588e+06
 2018-12-31    8.666820e+06
 2019-01-31    8.961709e+06
 2019-02-28    8.217292e+06
 2019-03-31    8.138389e+06
 2019-04-30    7.795594e+06
 2019-05-31    8.168687e+06
 2019-06-30    7.731522e+06
 2019-07-31    8.667542e+06
 2019-08-31    8.291768e+06
 2019-09-30    7.997535e+06
 2019-10-31    7.910404e+06
 2019-11-30    8.317907e+06
 2019-12-31    9.386332e+06
 2020-01-31    9.518785e+06
 2020-02-29    8.734695e+06
 2020-03-31    8.011420e+06
 2020-04-30    7.374809e+06
 2020-05-31    7.807453e+06
 2020-06-30    7.688209e+06
 2020-07-31    8.335269e+06
 2020-08-31    8.035135e+06
 2020-09-30    7.602747e+06
 2020-10-31    8.011133e+06
 2020-11-30    8.041630e+06

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.714341
p-value : 0.003916
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,1,0)(0,0,0)[1] intercept   : AIC=1191.589, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[1] intercept   : AIC=1190.197, Time=0.10 sec
 ARIMA(0,1,2)(0,0,0)[1] intercept   : AIC=1192.517, Time=0.06 sec
 ARIMA(0,1,3)(0,0,0)[1] intercept   : AIC=1192.624, Time=0.08 sec
 ARIMA(0,1,4)(0,0,0)[1] intercept   : AIC=1195.799, Time=0.07 sec
 ARIMA(0,1,5)(0,0,0)[1] intercept   : AIC=1198.921, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[1] intercept   : AIC=1190.091, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[1] intercept   : AIC=1192.197, Time=0.07 sec
 ARIMA(1,1,2)(0,0,0)[1] intercept   : AIC=1188.417, Time=0.16 sec
 ARIMA(1,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.33 sec
 ARIMA(1,1,4)(0,0,0)[1] intercept   : AIC=1191.352, Time=0.22 sec
 ARIMA(2,1,0)(0,0,0)[1] intercept   : AIC=1192.071, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[1] intercept   : AIC=1188.437, Time=0.15 sec
 ARIMA(2,1,2)(0,0,0)[1] intercept   : AIC=1189.949, Time=0.24 sec
 ARIMA(2,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.49 sec
 ARIMA(3,1,0)(0,0,0)

ARIMA(order=(3, 1, 2), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(3,1,2))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                   41
Model:                 ARIMA(3, 1, 2)   Log Likelihood                -583.989
Method:                       css-mle   S.D. of innovations         352177.447
Date:                Sat, 11 Sep 2021   AIC                           1181.977
Time:                        01:51:11   BIC                           1193.972
Sample:                             1   HQIC                          1186.345
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.294e+04   5040.150     -4.551      0.000   -3.28e+04   -1.31e+04
ar.L1.D.y      0.2443      0.276      0.886      0.376      -0.296       0.785
ar.L2.D.y      0.0895      0.180      0.497      0.6

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\base\model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarni

# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([7728271.39057048, 7755444.90808882]), array([352177.44653253, 380046.46146623]), array([[7038016.27919943, 8418526.50194152],
       [7010567.53116312, 8500322.28501451]]))
